In [1]:
import glob
import pandas as pd
import numpy as np
# 2 = Kaluzny Vband   3 = Kaluzny Bband

In [2]:
df = pd.read_csv('final_data_files/uncrowded_photometry.csv')#_clipped.csv'
#df_good = df[np.isfinite(df.resid_m4_3)]
#ids = df_good.id
df['mag_v'] = np.empty(len(df.id))
df['merr_v'] = np.empty(len(df.id))
df['mag_b'] = np.empty(len(df.id))
df['merr_b'] = np.empty(len(df.id))

In [3]:
dfcols = list(df.columns.values)
print dfcols

['id', 'mag_j', 'merr_j', 'type', 'per', 'mag_h', 'merr_h', 'mag_k', 'merr_k', 'mag_3', 'merr_3', 'mag_4', 'merr_4', 'mag_v', 'merr_v', 'mag_b', 'merr_b']


In [4]:
kdf = pd.read_csv('Kaluzny2004rrl.txt', delim_whitespace=True)

In [7]:
for i in glob.glob('Omega_Cen_phot/*.txt'):
    df1 = pd.read_csv(i, delim_whitespace=True, names=['mjd','mag','merr','band','uh'])
    starid = int(i.split('/')[-1].split('.')[0])
    v = df1[df1.band == 2]
    b = df1[df1.band == 3]
    vflux = np.mean(10**(v.mag / -2.5))
    vmag = -2.5 * np.log10(vflux)
    bflux = np.mean(10**(b.mag / -2.5))
    bmag = -2.5 * np.log10(bflux)
    verrsq = v.merr**2
    berrsq = b.merr**2
    verr = np.sqrt(verrsq.sum()) / np.sqrt(len(v.merr))
    berr = np.sqrt(berrsq.sum()) / np.sqrt(len(b.merr))
    if vmag == 0:
        vmag = np.nan
    if bmag == 0:
        bmag = np.nan
    if verr == 0:
        verr = np.nan
    if berr == 0:
        berr = np.nan
    if len(df.id[df.id == starid]) == 0:
        kdf_id = kdf[kdf.id == starid]
        newdf = pd.DataFrame(data=np.zeros((1,17)) + 99.99, columns=dfcols)
        newdf.id = starid
        newdf.per = float(kdf_id.per.values[0])
        newdf.type = float(kdf_id.type.values[0])
        newdf.mag_v = vmag
        newdf.mag_b = bmag
        newdf.merr_v = verr
        newdf.merr_b = berr
        newdf[newdf == 99.99] = np.nan
        df = df.append(newdf, ignore_index=True)
    else:
        df.mag_v[df.id == starid] = vmag
        df.mag_b[df.id == starid] = bmag
        df.merr_v[df.id == starid] = verr
        df.merr_b[df.id == starid] = berr

In [8]:
df.sort('id').to_csv('final_data_files/uncrowded_photometry_bv.csv', index=False)